In [3]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import env
import wrangle
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

## 1. Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.

In [4]:
# Using my get_zillow function in my wrangle.py file. 
# Will pull info from CodeUP database and put into a DataFrame.
df = wrangle.get_zillow()
df

,Unnamed: 0,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0,0.0,0.0,1776.0,440101.0,1947.0,5725.17,6037.0
1,1,4.0,4.0,3095.0,192544.0,2016.0,2460.72,6037.0
2,2,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
3,3,0.0,0.0,4053.0,511433.0,2002.0,6840.34,6037.0
4,4,0.0,0.0,1442.0,50689.0,NaN,1522.08,6037.0
...,...,...,...,...,...,...,...,...
1783868,1783868,4.0,4.0,4375.0,422400.0,2015.0,13877.56,6037.0
1783869,1783869,2.0,1.0,798.0,469300.0,2006.0,5764.45,6037.0
1783870,1783870,3.0,3.0,1526.0,594022.0,2014.0,7343.47,6037.0
1783871,1783871,4.0,4.0,2110.0,554009.0,2014.0,6761.20,6037.0


## 2. Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.

In [ ]:
df.shape

In [ ]:
df.info(null_counts=True)

In [ ]:
# Renaming columns for beter readability
df = df.rename(columns = {'bedroomcnt':'bedrooms', 
                          'bathroomcnt':'bathrooms', 
                          'calculatedfinishedsquarefeet':'area',
                          'taxvaluedollarcnt':'tax_value', 
                          'yearbuilt':'year_built'})

### Managing Nulls Values

In [ ]:
# getting total amount of nulls in each row

df.isnull().sum()

#### Takeaways
- 

## Visualizing Distributions and Outliers

In [ ]:
plt.figure(figsize=(16, 3))

# List of columns
cols = [col for col in df.columns if col not in ['fips', 'year_built']]

for i, col in enumerate(cols):

    # i starts at 0, but plot nos should start at 1
    plot_number = i + 1 

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display histogram for column.
    df[col].hist(bins=5)

    # Hide gridlines.
    plt.grid(False)
    
    # turn off scientific notation
    plt.ticklabel_format(useOffset=False)
    
plt.show()

### Takeaways
- Outliers must exist in each column for the x-axis to have the range shown
- tax_value is looking like it has severe skew, due to a x-axis range that goes to 1e8


In [ ]:
# Looking at tax_value in closer detail
df['tax_value'].hist(bins=100)

#### Now to look for outliers in a way that we can easily visualize them

In [ ]:
# We can utilize boxplots to show outliers for multiple features.

# List of columns
cols = [col for col in df.columns if col not in ['fips', 'year_built']]
plt.figure(figsize=(16, 20))
for i, col in enumerate(cols):

    # i starts at 0, but plot nos should start at 1
    plot_number = i + 1 

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display boxplot for column.
    sns.boxplot(data=df[col])

    # Hide gridlines.
    plt.grid(False)

plt.show()

#### Takeaways
- Substantial outliers exist
- If we presume that we are going to use our data to build a model that focuses on median home values, then we can drop these outliers.
- This may also help with our null value issue

In [ ]:
#### Now that we have seen the outliers we can go through and remove them.
def remove_outliers(df, k, col_list):
    ''' remove outliers from a list of columns in a dataframe 
        and return that dataframe
    '''
    
    for col in col_list:

        q1, q3 = df[col].quantile([.25, .75])  # get quartiles
        
        iqr = q3 - q1   # calculate interquartile range
        
        upper_bound = q3 + k * iqr   # get upper bound
        lower_bound = q1 - k * iqr   # get lower bound

        # return dataframe without outliers
        
        df = df[(df[col] > lower_bound) & (df[col] < upper_bound)]
        
    return df

In [ ]:
df = remove_outliers(df, 1.5, ['bedrooms', 'bathrooms', 'area', 'tax_value', 'taxamount'])
df

## Revisualize Distributions

In [ ]:
plt.figure(figsize=(16, 3))

# List of columns
cols = [col for col in df.columns if col not in ['fips', 'year_built']]

for i, col in enumerate(cols):

    # i starts at 0, but plot nos should start at 1
    plot_number = i + 1 

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display histogram for column.
    df[col].hist(bins=5)

    # Hide gridlines.
    plt.grid(False)
    
    # turn off scientific notation
    plt.ticklabel_format(useOffset=False)
    
    # mitigate overlap
    plt.tight_layout()
    
plt.show()

In [ ]:
# List of columns
cols = ['bedrooms', 'bathrooms', 'area', 'tax_value', 'taxamount']

plt.figure(figsize=(16, 3))

for i, col in enumerate(cols):

    # i starts at 0, but plot should start at 1
    plot_number = i + 1 

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display boxplot for column.
    sns.boxplot(data=df[[col]])

    # Hide gridlines.
    plt.grid(False)

    # sets proper spacing between plots
    plt.tight_layout()
    
plt.show()

In [ ]:
# get total of null values for each row
df.isnull().sum()

In [ ]:
# Checking the data types for each column
df.info()

In [ ]:
# get value counts and decide on data types
cols = df.columns

for col in cols:
    
    print(col.upper())
    print(df[col].value_counts())
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print()

In [ ]:
# Will be casting fips and year built as an object because it is a numerical represintation of a catagorical value

df.fips = df.fips.astype(object)
df.year_built = df.year_built.astype(object)

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
### Splitting the data

train_validate, test = train_test_split(df, test_size=.2, random_state=123)
train, validate = train_test_split(train_validate, test_size=.3, random_state=123)

In [ ]:
# To deal with the remainding nulls in my dataframe I will be imputing the nulls with the median year_built value.

imputer = SimpleImputer(strategy='median')  # build imputer

imputer.fit(train[['year_built']]) # fit to train

# transform the data
train[['year_built']] = imputer.transform(train[['year_built']])
validate[['year_built']] = imputer.transform(validate[['year_built']])
test[['year_built']] = imputer.transform(test[['year_built']])

In [ ]:
train.isnull().sum()

In [ ]:
validate.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
df.head()

## 3. Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe with no missing values in your wrangle.py file. Name your final function wrangle_zillow.

In [ ]:
# After placing all of my previous work into a functions in my wrangle file
# I will now test it to ensure it produces the same dataframe as above.
wrangle.wrangle_zillow()